In [1]:
# Imports
import re
import json
import requests
import folium
import pykx as kx

In [2]:
conn = kx.SyncQConnection('localhost', 5114)
ships = conn('select from latest[] where destination like "FIS*",speed>=0').py()

In [3]:
names = [ships[x]['shipname'] for x in ships]
coords = [(ships[x]['lat'],ships[x]['lon']) for x in ships]
shipsdict = {k:v for (k,v) in zip(names,coords)}

In [4]:
# Create map
map = folium.Map((65.585, 22.155), zoom_start=5)
for ship in shipsdict:
        folium.Marker(location=shipsdict[ship], tooltip=ship.decode('utf-8')).add_to(map)
map

In [5]:
conn = kx.SyncQConnection('localhost', 5114)
phist = conn('`receivets xdesc positionHist[257003340;2025.07.07D0;2025.07.08D0]').py()
coords = [(x,y) for (x,y) in zip(phist['lat'],phist['lon'])]
times = [x.isoformat() for x in phist['receivets']]
positions = {k:v for (k,v) in zip(times,coords)}
paths = [(x,y) for (x,y) in zip(coords[1:],coords[:-1])]

In [6]:
# Create map
map = folium.Map(coords[int(len(coords)/2)], zoom_start=13)
for p in positions: 
        folium.Marker(location=positions[p], tooltip=p, color='red').add_to(map)
for p in paths: 
        folium.PolyLine(locations=p).add_to(map)
folium.Marker(location=coords[-1], tooltip='START: '+times[-1], 
              icon=folium.Icon(icon='play', color='green')).add_to(map)
folium.Marker(location=coords[1], tooltip='FINISH: '+times[1], 
              icon=folium.Icon(icon='stop', color='red')).add_to(map)
map